# Multi-Objective Algorithm

In this notebook, we will use the Reference Vector Guided Evolutionary Algorithm (**RVEA**) to find the optimal solutions of the **DTLZ2** problem.

In [1]:
import time
import torch
from torch.profiler import ProfilerActivity, profile

from evox.algorithms import RVEA
from evox.metrics import igd
from evox.problems.numerical import DTLZ2
from evox.workflows import StdWorkflow

## (Optional) Use GPU to run the code
We often prefer to run our code on a GPU for faster execution. However, if a GPU is unavailable, running on a CPU is also acceptable.

In [2]:
# Use GPU first to run the code.
torch.set_default_device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.get_default_device())

cuda:0


## Running example: RVEA on DTLZ2 problem
The following code is used to set up the [`DTLZ2`](#evox.problems.numerical.dtlz) problem and the [`RVEA`](#evox.algorithms.mo.rvea) algorithm. More information about the problem and algorithm can be found in the corresponding section of the documentation.

In [3]:
# Init the problem, algorithm and workflow.
prob = DTLZ2(m=3)
pf = prob.pf()
algo = RVEA(pop_size=100, n_objs=3, lb=-torch.zeros(12), ub=torch.ones(12))
workflow = StdWorkflow()
workflow.setup(algo, prob)
workflow.init_step()

With this setup in place, we can now start to optimize. We set to let the multi-objective algorithm optimize for 100 steps on this problem

In [4]:
# Run the workflow for 100 steps
t = time.time()
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, profile_memory=True) as prof:
    for i in range(100):
        workflow.step()
        fit = workflow.algorithm.fit
        fit = fit[~torch.isnan(fit).any(dim=1)]
        if i % 10 == 0:
          print(igd(fit, pf))

tensor(0.4746, device='cuda:0')
tensor(0.1650, device='cuda:0')
tensor(0.0902, device='cuda:0')
tensor(0.0653, device='cuda:0')
tensor(0.0586, device='cuda:0')
tensor(0.0564, device='cuda:0')
tensor(0.0559, device='cuda:0')
tensor(0.0551, device='cuda:0')
tensor(0.0546, device='cuda:0')
tensor(0.0544, device='cuda:0')


(Optional) You can uncomment and execute the following code to print a table to view the hotspot functions.

In [ ]:
# print(prof.key_averages().table())
# torch.cuda.synchronize()
# print(time.time() - t)